# Chapter 12: Delegates, Events, and Lambda Expressions

So far, you've worked with methods as members of classes – blocks of code that you call directly. But what if you need to pass a method as a parameter to another method? Or store a reference to a method so you can call it later? This is where **delegates** come in. Delegates are objects that reference methods, enabling you to treat methods as first‑class citizens.

Building on delegates, **events** provide a standardized way to implement the publisher/subscriber pattern, where an object notifies other objects when something happens. And **lambda expressions** give you a concise syntax to create delegates inline.

In this chapter, you'll learn:

- What delegates are and how to declare and use them.
- The built‑in delegate types: `Action`, `Func`, and `Predicate`.
- **Multicast delegates** – delegates that can call multiple methods.
- **Anonymous methods** and **lambda expressions** as a shorthand.
- How to declare and raise **events**.
- The `EventHandler` delegate and custom event arguments.
- Best practices for events, including thread safety and memory leaks.
- A practical example that ties everything together.

By the end, you'll be able to create flexible, decoupled architectures using delegates and events, and write concise inline code with lambdas.

---

## 12.1 What Are Delegates?

A **delegate** is a type that represents references to methods with a specific parameter list and return type. When you instantiate a delegate, you can associate it with any method that has a compatible signature. You can then call the method through the delegate.

Think of a delegate as a **type‑safe function pointer**. They are the foundation of events and callbacks in C#.

### Declaring a Delegate

You declare a delegate using the `delegate` keyword, followed by a method signature.

```csharp
public delegate int Operation(int x, int y);
```

This defines a delegate type named `Operation` that can reference any method that takes two `int` parameters and returns an `int`.

### Using a Delegate

```csharp
public class Math
{
    public static int Add(int a, int b) => a + b;
    public static int Multiply(int a, int b) => a * b;
}

// Create delegate instances
Operation op1 = new Operation(Math.Add);
Operation op2 = Math.Multiply; // simplified syntax

// Invoke the delegate
int result1 = op1(5, 3);        // 8
int result2 = op2(5, 3);        // 15

Console.WriteLine(result1);
Console.WriteLine(result2);
```

You can also use the `Invoke` method explicitly: `op1.Invoke(5, 3)`.

### Why Delegates?

Delegates enable methods to be passed as parameters. This is useful for:

- **Callbacks** – execute a method after an operation completes.
- **Event handling** – notify subscribers when something happens.
- **LINQ** – many LINQ methods take delegates (e.g., `Where` expects a `Func<TSource, bool>`).
- **Strategy pattern** – encapsulate an algorithm and make it interchangeable.

---

## 12.2 Built‑in Delegate Types: `Action`, `Func`, and `Predicate`

Instead of declaring custom delegates for common patterns, C# provides generic delegate types in the `System` namespace.

### `Action`

`Action` represents a delegate that returns `void` and can take up to 16 parameters.

```csharp
Action greet = () => Console.WriteLine("Hello!");
Action<string> greetWithName = (name) => Console.WriteLine($"Hello, {name}!");

greet();
greetWithName("Alice");
```

### `Func`

`Func` represents a delegate that returns a value. The last type parameter is the return type; the preceding ones are the parameter types.

```csharp
Func<int, int, int> add = (x, y) => x + y;
Func<int, int> square = x => x * x;

Console.WriteLine(add(3, 4));   // 7
Console.WriteLine(square(5));    // 25
```

### `Predicate`

`Predicate<T>` is a delegate that takes one parameter and returns a `bool`. It's equivalent to `Func<T, bool>`, but is often used for conditions.

```csharp
Predicate<int> isEven = x => x % 2 == 0;
Console.WriteLine(isEven(4)); // True
```

In practice, you'll see `Func<T, bool>` used more often than `Predicate`, but both exist.

These built‑in delegates cover most needs, so you rarely need to declare your own delegate types.

---

## 12.3 Multicast Delegates

Delegates can reference **multiple methods**. This is called a **multicast delegate**. The `+` and `+=` operators combine delegates, and `-` and `-=` remove them.

```csharp
Action notify = () => Console.WriteLine("Notification 1");
notify += () => Console.WriteLine("Notification 2");
notify += () => Console.WriteLine("Notification 3");

notify(); // Calls all three methods in order
```

When you invoke a multicast delegate, each method is called in the order they were added. If any method throws an exception, subsequent methods may not be called (depending on the delegate type – for `Action`, they are; for custom delegates, you need to handle manually).

**Important:** Multicast delegates work for delegates that return `void`. If the delegate has a return type, only the last invoked method's return value is returned; the others are ignored. For this reason, multicast delegates are typically used with `void` delegates.

### Example: Combining Delegates

```csharp
public class Logger
{
    public static void LogToConsole(string message) => Console.WriteLine($"Console: {message}");
    public static void LogToFile(string message) => Console.WriteLine($"File: {message}"); // simulate
}

Action<string> log = Logger.LogToConsole;
log += Logger.LogToFile;

log("Application started");
```

---

## 12.4 Anonymous Methods and Lambda Expressions

Often, you need a small method that you'll only use once. Instead of defining a separate named method, you can use an **anonymous method** or a **lambda expression**.

### Anonymous Methods (C# 2.0)

An anonymous method is defined using the `delegate` keyword inline.

```csharp
Func<int, int, int> add = delegate (int a, int b) { return a + b; };
Console.WriteLine(add(3, 4));
```

### Lambda Expressions (C# 3.0 and later)

Lambdas are a more concise way to write anonymous methods. They use the `=>` operator.

```csharp
Func<int, int, int> add = (a, b) => a + b;
```

If there's only one parameter, you can omit parentheses:

```csharp
Func<int, int> square = x => x * x;
```

If the lambda has multiple statements, use curly braces and a `return` statement:

```csharp
Action<string> greet = name =>
{
    string message = $"Hello, {name}!";
    Console.WriteLine(message);
};
```

Lambdas are heavily used in LINQ:

```csharp
List<int> numbers = new List<int> { 1, 2, 3, 4, 5 };
var evens = numbers.Where(x => x % 2 == 0);
```

### Capturing Variables (Closures)

Lambdas can capture variables from the enclosing scope. This is called a **closure**.

```csharp
int factor = 2;
Func<int, int> multiplier = x => x * factor;
Console.WriteLine(multiplier(5)); // 10
```

The lambda "remembers" the value of `factor`. Be cautious: if `factor` changes before the lambda is invoked, it sees the current value.

---

## 12.5 Events – The Publisher/Subscriber Pattern

An **event** is a special kind of delegate that enforces the publisher/subscriber pattern. The **publisher** class declares the event; **subscribers** attach methods to it. When the event is raised, all subscriber methods are called.

### Declaring an Event

You declare an event using the `event` keyword followed by a delegate type.

```csharp
public class Button
{
    // Declare an event
    public event EventHandler Click;

    public void SimulateClick()
    {
        // Raise the event (if there are subscribers)
        Click?.Invoke(this, EventArgs.Empty);
    }
}
```

Here, `EventHandler` is a built‑in delegate defined as:

```csharp
public delegate void EventHandler(object? sender, EventArgs e);
```

It's the standard pattern for events in .NET: the sender is the object raising the event, and `EventArgs` contains any additional data.

### Subscribing to an Event

```csharp
Button button = new Button();
button.Click += Button_Click; // subscribe

private void Button_Click(object? sender, EventArgs e)
{
    Console.WriteLine("Button was clicked!");
}
```

You can also use a lambda:

```csharp
button.Click += (sender, e) => Console.WriteLine("Clicked!");
```

To unsubscribe, use `-=` with the same handler.

### Raising Events Safely

Always check for `null` before raising, because if there are no subscribers, the event is `null`. The `?.Invoke()` syntax (null‑conditional operator) does this safely and atomically.

### Custom Event Arguments

If you need to pass data with the event, create a class derived from `EventArgs`.

```csharp
public class TemperatureChangedEventArgs : EventArgs
{
    public double NewTemperature { get; set; }
    public double OldTemperature { get; set; }
}

public class Thermostat
{
    private double _temperature;
    public event EventHandler<TemperatureChangedEventArgs> TemperatureChanged;

    public double Temperature
    {
        get => _temperature;
        set
        {
            if (_temperature != value)
            {
                double old = _temperature;
                _temperature = value;
                OnTemperatureChanged(old, value);
            }
        }
    }

    protected virtual void OnTemperatureChanged(double oldTemp, double newTemp)
    {
        TemperatureChanged?.Invoke(this, new TemperatureChangedEventArgs 
        { 
            OldTemperature = oldTemp, 
            NewTemperature = newTemp 
        });
    }
}
```

Subscribers can then access the data:

```csharp
thermostat.TemperatureChanged += (sender, e) =>
{
    Console.WriteLine($"Temperature changed from {e.OldTemperature} to {e.NewTemperature}");
};
```

---

## 12.6 EventHandler and EventHandler<T>

The .NET framework provides two standard delegate types for events:

- `EventHandler` – for events with no data.
- `EventHandler<TEventArgs>` – for events with data, where `TEventArgs` derives from `EventArgs`.

Using these ensures consistency across your code and with framework events.

---

## 12.7 Best Practices for Delegates and Events

### 1. Use the Built‑in Types

Prefer `Action`, `Func`, `EventHandler`, and `EventHandler<T>` over custom delegate types unless you have a specific reason.

### 2. Check for Null Before Raising Events

Always use `EventName?.Invoke(...)` to avoid `NullReferenceException`.

### 3. Make Events Virtual

If your class is designed for inheritance, consider making the event‑raising method `protected virtual` so derived classes can override behavior.

### 4. Be Careful with Multicast Delegate Return Values

If you need each subscriber to return a value, use a different pattern (e.g., a list of delegates) because multicast delegates only return the last value.

### 5. Avoid Memory Leaks with Events

When a subscriber subscribes to an event, the publisher holds a reference to the subscriber. If the subscriber goes out of scope but doesn't unsubscribe, it remains alive (garbage collector can't collect it). Always unsubscribe when no longer needed, especially in long‑lived objects.

```csharp
button.Click -= Button_Click; // important for cleanup
```

### 6. Use `nameof` for Event Names

When raising events, use `nameof(EventName)` in any logging or debugging to avoid magic strings.

### 7. Keep Lambdas Short

Lambdas are great for small logic. If a lambda becomes long or complex, consider extracting it to a named method for readability.

### 8. Thread Safety

If events can be raised from multiple threads, you need to ensure thread safety. The null‑conditional `?.Invoke()` is atomic, but if you need more (like copying the delegate before invoking), do:

```csharp
var handler = TemperatureChanged;
if (handler != null)
    handler(this, e);
```

This avoids a race condition where the event becomes null after the check.

---

## 12.8 Putting It All Together: A Message Broadcaster

Let's build a simple message broadcasting system using delegates and events. We'll have a `Broadcaster` class that raises an event when a message is received, and multiple subscribers that handle the message.

```csharp
using System;

namespace EventDemo
{
    // Custom event args
    public class MessageEventArgs : EventArgs
    {
        public string Message { get; set; }
        public string Sender { get; set; }
    }

    // Publisher class
    public class MessageBroadcaster
    {
        // Event using generic EventHandler
        public event EventHandler<MessageEventArgs> MessageReceived;

        public void BroadcastMessage(string sender, string message)
        {
            Console.WriteLine($"[Broadcaster] Broadcasting message from {sender}...");
            OnMessageReceived(sender, message);
        }

        protected virtual void OnMessageReceived(string sender, string message)
        {
            MessageReceived?.Invoke(this, new MessageEventArgs 
            { 
                Sender = sender, 
                Message = message 
            });
        }
    }

    // Subscriber classes
    public class EmailLogger
    {
        public void Subscribe(MessageBroadcaster broadcaster)
        {
            broadcaster.MessageReceived += OnMessageReceived;
        }

        public void Unsubscribe(MessageBroadcaster broadcaster)
        {
            broadcaster.MessageReceived -= OnMessageReceived;
        }

        private void OnMessageReceived(object sender, MessageEventArgs e)
        {
            Console.WriteLine($"[EmailLogger] Sending email: {e.Sender} says '{e.Message}'");
        }
    }

    public class ConsoleLogger
    {
        public void Subscribe(MessageBroadcaster broadcaster)
        {
            // Using lambda for subscription
            broadcaster.MessageReceived += (s, e) =>
            {
                Console.WriteLine($"[ConsoleLogger] {e.Sender}: {e.Message}");
            };
        }
    }

    public class FilteredSubscriber
    {
        private readonly string _keyword;

        public FilteredSubscriber(string keyword)
        {
            _keyword = keyword;
        }

        public void Subscribe(MessageBroadcaster broadcaster)
        {
            broadcaster.MessageReceived += OnMessageReceived;
        }

        private void OnMessageReceived(object sender, MessageEventArgs e)
        {
            if (e.Message.Contains(_keyword, StringComparison.OrdinalIgnoreCase))
            {
                Console.WriteLine($"[FilteredSubscriber] Found '{_keyword}': {e.Message}");
            }
        }
    }

    class Program
    {
        static void Main()
        {
            var broadcaster = new MessageBroadcaster();

            // Create subscribers
            var emailLogger = new EmailLogger();
            var consoleLogger = new ConsoleLogger();
            var alertSubscriber = new FilteredSubscriber("urgent");

            // Subscribe
            emailLogger.Subscribe(broadcaster);
            consoleLogger.Subscribe(broadcaster);
            alertSubscriber.Subscribe(broadcaster);

            // Broadcast messages
            broadcaster.BroadcastMessage("Alice", "Hello everyone!");
            broadcaster.BroadcastMessage("System", "This is an urgent alert");
            broadcaster.BroadcastMessage("Bob", "Meeting at 3pm");

            // Unsubscribe one
            emailLogger.Unsubscribe(broadcaster);

            Console.WriteLine("\nAfter unsubscribing email logger:");
            broadcaster.BroadcastMessage("Charlie", "Another message");

            // Demonstrate delegate use with lambda
            Console.WriteLine("\nUsing Func delegate:");
            Func<int, int, int> add = (a, b) => a + b;
            Console.WriteLine($"5 + 3 = {add(5, 3)}");

            // Multicast Action
            Action<string> logActions = Console.WriteLine;
            logActions += (msg) => System.Diagnostics.Debug.WriteLine(msg);
            logActions("This goes to console and debug output");
        }
    }
}
```

**Explanation:**

- `MessageBroadcaster` is the publisher, with an event `MessageReceived` using `EventHandler<MessageEventArgs>`.
- `MessageEventArgs` carries the sender and message.
- Subscribers (`EmailLogger`, `ConsoleLogger`, `FilteredSubscriber`) attach handlers. They can be instance methods or lambdas.
- `FilteredSubscriber` demonstrates conditional handling.
- The `Main` method shows subscribing, broadcasting, and unsubscribing.
- We also show a simple `Func` delegate and a multicast `Action` to reinforce delegate concepts.

This example illustrates how events decouple the broadcaster from the subscribers – the broadcaster doesn't know anything about the subscribers, yet they all receive messages.

---

## 12.9 Common Pitfalls and Best Practices – Recap

- **Delegate vs. Event** – An event is a wrapper around a delegate that only allows `+=` and `-=` from outside. It prevents subscribers from clearing other subscriptions or invoking the delegate directly.
- **Memory Leaks** – Always unsubscribe from events when the subscriber is no longer needed, especially in long‑running applications.
- **Exception Handling** – If a subscriber throws an exception, it may prevent other subscribers from being notified. Consider wrapping subscriber calls in try‑catch if you need robustness.
- **Lambda Variable Capture** – Be aware that lambdas capture variables, which can extend the lifetime of objects. This can also cause memory leaks if not careful.
- **Use `EventHandler<T>` for Consistency** – It's the standard pattern; follow it unless you have a compelling reason not to.

---

## 12.10 Chapter Summary

In this chapter, you've mastered delegates, events, and lambda expressions – key features for writing flexible and decoupled code.

- **Delegates** are type‑safe function pointers that allow methods to be passed as parameters.
- **Built‑in delegates** `Action`, `Func`, and `Predicate` cover most needs.
- **Multicast delegates** can invoke multiple methods.
- **Anonymous methods and lambdas** provide concise inline syntax.
- **Events** implement the publisher/subscriber pattern, with `EventHandler` as the standard.
- **Custom event arguments** let you pass data with events.
- Best practices ensure your event‑driven code is robust and leak‑free.

With these tools, you can design systems where components communicate without tight coupling, making your code more modular and maintainable.

In the next chapter, **Generics – Code Reusability with Type Safety**, we'll explore how generics allow you to write classes and methods that work with any type while preserving type safety. You'll learn about generic classes, methods, constraints, and covariance/contravariance.

**Exercises:**

1. Write a method that takes a `List<int>` and a `Predicate<int>` and returns a new list with elements that satisfy the predicate. Test it with different predicates.
2. Create a simple `Timer` class that raises an event every second. Use `EventHandler` and test with multiple subscribers.
3. Implement a custom `Sort` method that takes a `List<T>` and a `Comparison<T>` delegate (which returns an int) and sorts using that comparison. Test with sorting strings by length.
4. Build a simple event-driven calculator: a class `Calculator` that raises events for each operation (Add, Subtract, etc.) and a logger that subscribes and logs operations.
5. Experiment with lambda captures: create a list of `Action` delegates inside a loop and see what happens when you invoke them later. Understand why you might need to copy the loop variable.

Now, get ready to dive into generics in Chapter 13!

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='../2. object_oriented_programming_the_c_way/11. encapsulation_best_practices.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='13. generics_code_reusability_with_type_safety.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
